# Spaceship Titanic - Complete Pipeline

**Competition:** Spaceship Titanic (Kaggle)  
**Problem:** Binary Classification (Transported: True/False)  
**Metric:** Accuracy  
**Data:** ~8700 train / ~4300 test / 14 features  
**Strategy:** Feature Engineering + LightGBM + XGBoost + CatBoost Ensemble

## 1. Configuration & Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier, Pool
import optuna
from scipy.stats import rankdata
import warnings
import gc
import os

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)
sns.set_style('whitegrid')

SEED = 42
N_FOLDS = 10
TARGET = 'Transported'

def seed_everything(seed=SEED):
    import random
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything()

print('Setup complete.')

## 2. Data Loading

In [ ]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
sample_sub = pd.read_csv('../data/sample_submission.csv')

print(f'Train shape: {train.shape}')
print(f'Test shape:  {test.shape}')
print(f'Submission:  {sample_sub.shape}')
print(f'\nTarget distribution:')
print(train[TARGET].value_counts(normalize=True))
train.head()

## 3. EDA - Exploratory Data Analysis

In [ ]:
print('=== DATA INFO ===')
print(f'\nColumns ({len(train.columns)}):')
for col in train.columns:
    dtype = train[col].dtype
    nunique = train[col].nunique()
    null_pct = train[col].isnull().mean() * 100
    print(f'  {col:20s} | {str(dtype):10s} | {nunique:6d} unique | {null_pct:5.2f}% missing')

In [ ]:
# Missing values comparison train vs test
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

train_miss = train.isnull().mean().sort_values(ascending=False) * 100
test_miss = test.isnull().mean().sort_values(ascending=False) * 100

train_miss[train_miss > 0].plot(kind='barh', ax=axes[0], color='steelblue')
axes[0].set_title('Train - Missing %')
axes[0].set_xlabel('%')

test_miss[test_miss > 0].plot(kind='barh', ax=axes[1], color='coral')
axes[1].set_title('Test - Missing %')
axes[1].set_xlabel('%')

plt.tight_layout()
plt.show()

In [ ]:
# Target vs categorical features
cat_cols_eda = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
for ax, col in zip(axes.flatten(), cat_cols_eda):
    ct = pd.crosstab(train[col], train[TARGET], normalize='index') * 100
    ct.plot(kind='bar', stacked=True, ax=ax, colormap='RdYlGn')
    ax.set_title(f'{col} vs Transported')
    ax.set_ylabel('% Transported')
    ax.legend(title='Transported')
    ax.tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

In [ ]:
# Spending columns distribution by Transported
spend_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

fig, axes = plt.subplots(1, 5, figsize=(20, 4))
for ax, col in zip(axes, spend_cols):
    for transported in [True, False]:
        subset = train[train[TARGET] == transported][col].dropna()
        # Log transform for better visualization
        subset_log = np.log1p(subset)
        ax.hist(subset_log, bins=30, alpha=0.5, label=str(transported), density=True)
    ax.set_title(col)
    ax.legend(title='Transported', fontsize=8)
    ax.set_xlabel('log1p(amount)')

plt.suptitle('Spending Distributions by Transported (log scale)', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Age distribution by Transported
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for transported in [True, False]:
    subset = train[train[TARGET] == transported]['Age'].dropna()
    axes[0].hist(subset, bins=40, alpha=0.5, label=str(transported), density=True)
axes[0].set_title('Age Distribution by Transported')
axes[0].legend(title='Transported')
axes[0].set_xlabel('Age')

# Cabin Deck extraction for quick look
train_temp = train.copy()
train_temp['Deck'] = train_temp['Cabin'].str.split('/').str[0]
ct = pd.crosstab(train_temp['Deck'], train_temp[TARGET], normalize='index') * 100
ct[True].sort_values().plot(kind='barh', ax=axes[1], color='seagreen')
axes[1].set_title('% Transported by Deck')
axes[1].set_xlabel('% Transported')

plt.tight_layout()
plt.show()
del train_temp

In [ ]:
# Key EDA insights
print('=== KEY EDA INSIGHTS ===')
print()
print('1. CryoSleep is the STRONGEST predictor (~80% transported if True)')
print('2. Europa passengers more likely transported than Earth/Mars')
print('3. CryoSleep passengers have 0 spending (logical: they are asleep)')
print('4. Younger passengers (<18) slightly more likely transported')
print('5. Cabin Deck B & C have highest transport rates')
print('6. VIP passengers LESS likely transported')
print('7. Higher spending generally correlates with NOT transported')
print('8. PassengerId encodes group structure (XXXX_YY = group_member)')
print('9. ~2% missing values across most columns - manageable')
print('10. Target is well balanced (~50/50)')

## 4. Feature Engineering

In [ ]:
# Combine train+test for consistent feature engineering
train['is_train'] = 1
test['is_train'] = 0
test[TARGET] = np.nan

df = pd.concat([train, test], axis=0, ignore_index=True)
print(f'Combined shape: {df.shape}')

In [ ]:
spend_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

def feature_engineering(df):
    """Complete feature engineering pipeline."""
    
    # =========================================================
    # 4.1 - PassengerId decomposition
    # =========================================================
    df['Group'] = df['PassengerId'].str.split('_').str[0].astype(int)
    df['MemberNum'] = df['PassengerId'].str.split('_').str[1].astype(int)
    df['GroupSize'] = df.groupby('Group')['PassengerId'].transform('count')
    df['IsAlone'] = (df['GroupSize'] == 1).astype(int)
    
    # =========================================================
    # 4.2 - Cabin decomposition
    # =========================================================
    df['Deck'] = df['Cabin'].str.split('/').str[0]
    df['CabinNum'] = df['Cabin'].str.split('/').str[1].astype(float)
    df['Side'] = df['Cabin'].str.split('/').str[2]
    
    # =========================================================
    # 4.3 - Name features
    # =========================================================
    df['Surname'] = df['Name'].str.split().str[-1]
    df['FamilySize'] = df.groupby('Surname')['PassengerId'].transform('count')
    # Handle NaN surnames
    df.loc[df['Surname'].isna(), 'FamilySize'] = 1
    
    # =========================================================
    # 4.4 - Spending features
    # =========================================================
    df['TotalSpend'] = df[spend_cols].sum(axis=1)
    df['NoSpend'] = (df['TotalSpend'] == 0).astype(int)
    df['NumServicesUsed'] = (df[spend_cols] > 0).sum(axis=1)
    
    # Luxury vs basic spending
    df['LuxurySpend'] = df['Spa'] + df['VRDeck'] + df['RoomService']
    df['BasicSpend'] = df['FoodCourt'] + df['ShoppingMall']
    
    # Spending ratios (avoid division by zero)
    df['LuxuryRatio'] = df['LuxurySpend'] / (df['TotalSpend'] + 1)
    df['BasicRatio'] = df['BasicSpend'] / (df['TotalSpend'] + 1)
    
    # Max single spending category
    df['MaxSpend'] = df[spend_cols].max(axis=1)
    df['MaxSpendCategory'] = df[spend_cols].idxmax(axis=1)
    
    # Log transforms of spending
    for col in spend_cols:
        df[f'{col}_log'] = np.log1p(df[col])
    df['TotalSpend_log'] = np.log1p(df['TotalSpend'])
    
    # =========================================================
    # 4.5 - Boolean features
    # =========================================================
    df['CryoSleep'] = df['CryoSleep'].map({True: 1, False: 0, 'True': 1, 'False': 0})
    df['VIP'] = df['VIP'].map({True: 1, False: 0, 'True': 1, 'False': 0})
    
    # =========================================================
    # 4.6 - Age features
    # =========================================================
    df['AgeGroup'] = pd.cut(df['Age'], bins=[0, 5, 12, 18, 30, 50, 80], 
                            labels=[0, 1, 2, 3, 4, 5]).astype(float)
    df['IsChild'] = (df['Age'] < 18).astype(float)
    df['IsElder'] = (df['Age'] >= 50).astype(float)
    
    # =========================================================
    # 4.7 - Interaction features
    # =========================================================
    df['CryoSleep_NoSpend'] = ((df['CryoSleep'] == 1) & (df['TotalSpend'] == 0)).astype(int)
    df['Age_TotalSpend'] = df['Age'] * df['TotalSpend_log']
    
    # =========================================================
    # 4.8 - Group-level features (inspired by titanicv2)
    # =========================================================
    # Average spending per group
    df['GroupSpend_mean'] = df.groupby('Group')['TotalSpend'].transform('mean')
    df['GroupSpend_max'] = df.groupby('Group')['TotalSpend'].transform('max')
    df['GroupAge_mean'] = df.groupby('Group')['Age'].transform('mean')
    
    # Cabin density
    df['CabinNum_rounded'] = (df['CabinNum'] // 10).astype(float)
    
    # =========================================================
    # 4.9 - Encode categoricals
    # =========================================================
    label_cols = ['HomePlanet', 'Destination', 'Deck', 'Side', 'MaxSpendCategory']
    label_encoders = {}
    for col in label_cols:
        le = LabelEncoder()
        df[col + '_le'] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le
    
    # Frequency encodings
    for col in ['HomePlanet', 'Destination', 'Deck', 'Side', 'Surname']:
        freq = df[col].value_counts(normalize=True)
        df[col + '_freq'] = df[col].map(freq)
    
    return df, label_encoders

df, label_encoders = feature_engineering(df)
print(f'Features after engineering: {df.shape[1]} columns')

In [ ]:
# =========================================================
# 4.10 - Smart imputation (after feature engineering)
# =========================================================

# CryoSleep passengers must have 0 spending
for col in spend_cols:
    mask = (df['CryoSleep'] == 1) & (df[col].isna())
    df.loc[mask, col] = 0

# Passengers with 0 total spend might be CryoSleep
mask = (df['CryoSleep'].isna()) & (df['TotalSpend'] == 0)
df.loc[mask, 'CryoSleep'] = 1
mask = (df['CryoSleep'].isna()) & (df['TotalSpend'] > 0)
df.loc[mask, 'CryoSleep'] = 0

# Fill remaining numericals with median
num_cols_fill = df.select_dtypes(include=[np.number]).columns
for col in num_cols_fill:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].median())

# Fill remaining categoricals with mode
cat_cols_fill = df.select_dtypes(include=['object']).columns
for col in cat_cols_fill:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].mode()[0])

print(f'Remaining nulls: {df.isnull().sum().sum()}')

In [ ]:
# =========================================================
# 4.11 - Group Survival Features (key trick from titanicv2)
# =========================================================

# Only use train data for group survival to avoid leakage
train_mask = df['is_train'] == 1

# Group survival rate (leave-one-out to avoid leakage within train)
train_df = df[train_mask].copy()

# For groups with >1 member: compute transported rate of OTHER members
group_sum = train_df.groupby('Group')[TARGET].transform('sum')
group_count = train_df.groupby('Group')[TARGET].transform('count')

# Leave-one-out: (sum - self) / (count - 1), safe division
denom = (group_count - 1).replace(0, np.nan)
train_df['GroupSurvival_loo'] = (group_sum - train_df[TARGET]) / denom
train_df['GroupSurvival_loo'] = train_df['GroupSurvival_loo'].fillna(0.5)  # no info for solo passengers

# For test: use full group transported rate from train
group_rate = train_df.groupby('Group')[TARGET].mean().to_dict()

test_df = df[~train_mask].copy()
test_df['GroupSurvival_loo'] = test_df['Group'].map(group_rate)
test_df['GroupSurvival_loo'] = test_df['GroupSurvival_loo'].fillna(0.5)

df = pd.concat([train_df, test_df], axis=0).sort_index()

# Same for surname (family survival)
train_df = df[train_mask].copy()
fam_sum = train_df.groupby('Surname')[TARGET].transform('sum')
fam_count = train_df.groupby('Surname')[TARGET].transform('count')
fam_denom = (fam_count - 1).replace(0, np.nan)
train_df['FamilySurvival_loo'] = (fam_sum - train_df[TARGET]) / fam_denom
train_df['FamilySurvival_loo'] = train_df['FamilySurvival_loo'].fillna(0.5)

fam_rate = train_df.groupby('Surname')[TARGET].mean().to_dict()
test_df = df[~train_mask].copy()
test_df['FamilySurvival_loo'] = test_df['Surname'].map(fam_rate)
test_df['FamilySurvival_loo'] = test_df['FamilySurvival_loo'].fillna(0.5)

df = pd.concat([train_df, test_df], axis=0).sort_index()

print('Group survival features added.')
print(f'GroupSurvival_loo stats: mean={df["GroupSurvival_loo"].mean():.3f}')
print(f'FamilySurvival_loo stats: mean={df["FamilySurvival_loo"].mean():.3f}')

In [ ]:
# Define final features
drop_cols = ['PassengerId', 'Name', 'Cabin', 'Surname', 'is_train', TARGET,
             'HomePlanet', 'Destination', 'Deck', 'Side', 'MaxSpendCategory']

features = [c for c in df.columns if c not in drop_cols]
print(f'Number of features: {len(features)}')
print(f'\nFeatures:')
for i, f in enumerate(sorted(features)):
    print(f'  {i+1:2d}. {f}')

In [ ]:
# Split back to train/test
train_df = df[df['is_train'] == 1].copy()
test_df = df[df['is_train'] == 0].copy()

X = train_df[features].values
y = train_df[TARGET].values.astype(int)
X_test = test_df[features].values

print(f'X: {X.shape}, y: {y.shape}, X_test: {X_test.shape}')
print(f'Target: {np.mean(y):.4f} (should be ~0.50)')

## 5. Modeling - LightGBM

In [ ]:
lgb_params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.03,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_child_samples': 20,
    'reg_alpha': 0.1,
    'reg_lambda': 1.0,
    'n_estimators': 5000,
    'verbose': -1,
    'n_jobs': -1,
    'random_state': SEED,
}

oof_lgb = np.zeros(len(X))
test_lgb = np.zeros(len(X_test))
fi_lgb = np.zeros(len(features))

kf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
    X_tr, X_val = X[train_idx], X[val_idx]
    y_tr, y_val = y[train_idx], y[val_idx]
    
    model = lgb.LGBMClassifier(**lgb_params)
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        callbacks=[lgb.early_stopping(200), lgb.log_evaluation(500)]
    )
    
    oof_lgb[val_idx] = model.predict_proba(X_val)[:, 1]
    test_lgb += model.predict_proba(X_test)[:, 1] / N_FOLDS
    fi_lgb += model.feature_importances_ / N_FOLDS
    
    fold_acc = accuracy_score(y_val, (oof_lgb[val_idx] > 0.5).astype(int))
    print(f'Fold {fold+1}/{N_FOLDS} - Accuracy: {fold_acc:.5f}')

lgb_acc = accuracy_score(y, (oof_lgb > 0.5).astype(int))
print(f'\nLightGBM CV Accuracy: {lgb_acc:.5f}')

In [ ]:
# Feature importance
fi_df = pd.DataFrame({'feature': features, 'importance': fi_lgb})
fi_df = fi_df.sort_values('importance', ascending=True).tail(25)

plt.figure(figsize=(10, 8))
plt.barh(fi_df['feature'], fi_df['importance'], color='steelblue')
plt.title(f'LightGBM Feature Importance (Top 25) - CV Acc: {lgb_acc:.5f}')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

## 6. Modeling - XGBoost

In [ ]:
xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 6,
    'learning_rate': 0.03,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 5,
    'reg_alpha': 0.1,
    'reg_lambda': 1.0,
    'n_estimators': 5000,
    'early_stopping_rounds': 200,
    'tree_method': 'hist',
    'random_state': SEED,
    'verbosity': 0,
}

oof_xgb = np.zeros(len(X))
test_xgb = np.zeros(len(X_test))

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
    X_tr, X_val = X[train_idx], X[val_idx]
    y_tr, y_val = y[train_idx], y[val_idx]
    
    model = xgb.XGBClassifier(**xgb_params)
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        verbose=500
    )
    
    oof_xgb[val_idx] = model.predict_proba(X_val)[:, 1]
    test_xgb += model.predict_proba(X_test)[:, 1] / N_FOLDS
    
    fold_acc = accuracy_score(y_val, (oof_xgb[val_idx] > 0.5).astype(int))
    print(f'Fold {fold+1}/{N_FOLDS} - Accuracy: {fold_acc:.5f}')

xgb_acc = accuracy_score(y, (oof_xgb > 0.5).astype(int))
print(f'\nXGBoost CV Accuracy: {xgb_acc:.5f}')

## 7. Modeling - CatBoost

In [ ]:
oof_cb = np.zeros(len(X))
test_cb = np.zeros(len(X_test))

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
    X_tr, X_val = X[train_idx], X[val_idx]
    y_tr, y_val = y[train_idx], y[val_idx]
    
    model = CatBoostClassifier(
        iterations=5000,
        learning_rate=0.03,
        depth=6,
        l2_leaf_reg=3.0,
        subsample=0.8,
        colsample_bylevel=0.8,
        min_data_in_leaf=20,
        random_seed=SEED,
        verbose=500,
        early_stopping_rounds=200,
        task_type='CPU',
    )
    
    model.fit(
        X_tr, y_tr,
        eval_set=(X_val, y_val),
    )
    
    oof_cb[val_idx] = model.predict_proba(X_val)[:, 1]
    test_cb += model.predict_proba(X_test)[:, 1] / N_FOLDS
    
    fold_acc = accuracy_score(y_val, (oof_cb[val_idx] > 0.5).astype(int))
    print(f'Fold {fold+1}/{N_FOLDS} - Accuracy: {fold_acc:.5f}')

cb_acc = accuracy_score(y, (oof_cb > 0.5).astype(int))
print(f'\nCatBoost CV Accuracy: {cb_acc:.5f}')

## 8. Ensemble

In [ ]:
print('=== Individual Model Scores ===')
print(f'LightGBM: {lgb_acc:.5f}')
print(f'XGBoost:  {xgb_acc:.5f}')
print(f'CatBoost: {cb_acc:.5f}')
print()

# Simple average ensemble
oof_avg = (oof_lgb + oof_xgb + oof_cb) / 3
avg_acc = accuracy_score(y, (oof_avg > 0.5).astype(int))
print(f'Simple Average Ensemble: {avg_acc:.5f}')

# Weighted ensemble - find optimal weights via grid search
best_acc = 0
best_w = (1/3, 1/3, 1/3)

for w1 in np.arange(0.1, 0.8, 0.05):
    for w2 in np.arange(0.1, 0.8 - w1, 0.05):
        w3 = 1 - w1 - w2
        if w3 < 0.05:
            continue
        oof_w = w1 * oof_lgb + w2 * oof_xgb + w3 * oof_cb
        acc = accuracy_score(y, (oof_w > 0.5).astype(int))
        if acc > best_acc:
            best_acc = acc
            best_w = (w1, w2, w3)

print(f'\nOptimal Weighted Ensemble: {best_acc:.5f}')
print(f'Weights: LGB={best_w[0]:.2f}, XGB={best_w[1]:.2f}, CB={best_w[2]:.2f}')

In [ ]:
# Rank-based ensemble (more robust to score scale differences)
oof_rank = (
    rankdata(oof_lgb) + rankdata(oof_xgb) + rankdata(oof_cb)
) / 3
rank_threshold = np.median(oof_rank)  # since ~50/50 split
rank_acc = accuracy_score(y, (oof_rank > rank_threshold).astype(int))
print(f'Rank Ensemble: {rank_acc:.5f}')

# Majority voting
votes = (
    (oof_lgb > 0.5).astype(int) + 
    (oof_xgb > 0.5).astype(int) + 
    (oof_cb > 0.5).astype(int)
)
vote_acc = accuracy_score(y, (votes >= 2).astype(int))
print(f'Majority Voting: {vote_acc:.5f}')

In [ ]:
# Select best ensemble method
ensemble_scores = {
    'Simple Average': avg_acc,
    'Weighted Average': best_acc,
    'Rank Ensemble': rank_acc,
    'Majority Voting': vote_acc,
}

best_method = max(ensemble_scores, key=ensemble_scores.get)
print(f'\nBest Ensemble Method: {best_method} ({ensemble_scores[best_method]:.5f})')

# Generate final test predictions with best method
if best_method == 'Simple Average':
    final_proba = (test_lgb + test_xgb + test_cb) / 3
elif best_method == 'Weighted Average':
    final_proba = best_w[0] * test_lgb + best_w[1] * test_xgb + best_w[2] * test_cb
elif best_method == 'Rank Ensemble':
    final_proba = (rankdata(test_lgb) + rankdata(test_xgb) + rankdata(test_cb)) / 3
    # Use same relative threshold
elif best_method == 'Majority Voting':
    final_votes = (
        (test_lgb > 0.5).astype(int) + 
        (test_xgb > 0.5).astype(int) + 
        (test_cb > 0.5).astype(int)
    )

# Convert to binary predictions
if best_method == 'Majority Voting':
    final_preds = (final_votes >= 2)
elif best_method == 'Rank Ensemble':
    final_preds = (final_proba > np.median(final_proba))
else:
    final_preds = (final_proba > 0.5)

print(f'\nTest predictions: {final_preds.sum()} True / {len(final_preds) - final_preds.sum()} False')
print(f'Ratio: {final_preds.mean():.4f} (should be ~0.50)')

## 9. Submission

In [ ]:
submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'].values,
    'Transported': final_preds
})

# Ensure boolean format matches sample_submission
submission['Transported'] = submission['Transported'].astype(bool)

submission.to_csv('../submissions/submission.csv', index=False)

print('Submission saved!')
print(f'Shape: {submission.shape}')
print(f'\nDistribution:')
print(submission['Transported'].value_counts(normalize=True))
print(f'\nFirst rows:')
submission.head(10)

In [ ]:
# Validation: compare with sample submission format
assert submission.shape[0] == sample_sub.shape[0], 'Row count mismatch!'
assert list(submission.columns) == list(sample_sub.columns), 'Column mismatch!'
assert submission['Transported'].dtype == bool, 'Type mismatch!'
print('Submission format validated!')

## 10. Summary & Next Steps

In [ ]:
print('=' * 60)
print('          SPACESHIP TITANIC - PIPELINE SUMMARY')
print('=' * 60)
print(f'\nFeatures engineered: {len(features)}')
print(f'CV Folds: {N_FOLDS}')
print(f'\n--- Model Scores (CV Accuracy) ---')
print(f'  LightGBM:  {lgb_acc:.5f}')
print(f'  XGBoost:   {xgb_acc:.5f}')
print(f'  CatBoost:  {cb_acc:.5f}')
print(f'\n--- Ensemble Scores ---')
for method, score in sorted(ensemble_scores.items(), key=lambda x: -x[1]):
    marker = ' <-- BEST' if method == best_method else ''
    print(f'  {method:20s}: {score:.5f}{marker}')
print(f'\n--- Submission ---')
print(f'  File: ../submissions/submission.csv')
print(f'  Rows: {submission.shape[0]}')
print(f'  True ratio: {submission["Transported"].mean():.4f}')
print(f'\n--- Next Steps ---')
print('  1. Submit to Kaggle and check LB score')
print('  2. Hyperparameter tuning with Optuna')
print('  3. Add more interaction features')
print('  4. Try neural network (TabNet, MLP)')
print('  5. Pseudo-labeling with confident predictions')
print('  6. Target encoding with proper CV')
print('=' * 60)